In [0]:
# remove an existing checkpoint if created with a different model
!rm -rf checkpoint


This colab sheet is an RPS training playground. 
For it to work, runtime has to be supported by GPU with suffiicent memory (does not happen often on free tier)

In [0]:
# content snippet for style to be applied to

input = """She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her. The window frame trembled with the speed of the motion, the pane hung over empty
darkness, and dots of light slashed across the glass as luminous streaks, once in a while.

Her leg, sculptured by the tight sheen of the stocking, its long line running straight, over an arched
instep, to the tip of a foot in a high-heeled pump, had a feminine elegance that seemed out of place in
the dusty train car and oddly incongruous with the rest of her. She wore a battered camel's hair coat
that had been expensive, wrapped shapelessly about her slender, nervous body. The coat collar was
raised to the slanting brim of her hat. A sweep of brown hair fell back, almost touching the line of her
shoulders. Her face was made of angular planes, the shape of her mouth clear-cut, a sensual mouth
held closed with inflexible precision. She kept her hands in the coat pockets, her posture taut, as if
she resented immobility, and unfeminine, as if she were unconscious of her own body and that it was
a woman's body. She sat listening to the music. It was a symphony of triumph. The notes flowed up,
they spoke of rising and they were the rising itself, they were the essence and the form of upward
motion, they seemed to embody every human act and thought that had ascent as its motive. It was a
sunburst of sound, breaking out of hiding and spreading open. It had the freedom of release and the
tension of purpose. It swept space clean, and left nothing but the joy of an unobstructed effort. Only a
faint echo within the sounds spoke of that from which the music had escaped, but spoke in laughing
astonishment at the discovery that there was no ugliness or pain, and there never had had to be. It was
the song of an immense deliverance.



She thought: For just a few moments — while this lasts — it is all right to surrender completely — to
forget everything and just permit yourself to feel. She thought: Let go — drop the controls — this is it.
Somewhere on the edge of her mind, under the music, she heard the sound of train wheels. They
knocked in an even rhythm, every fourth knock accented, as if stressing a conscious purpose. She
could relax, because she heard the wheels. She listened to the symphony, thinking: This is why the
wheels have to be kept going, and this is where they're going.

She had never heard that symphony before, but she knew that it was written by Richard Halley. She
recognized the violence and the magnificent intensity. She recognized the style of the theme; it was a
clear, complex melody — at a time when no one wrote melody any longer. . . . She sat looking up at the
ceiling of the car, but she did not see it and she had forgotten where she was. She did not know
whether she was hearing a frill symphony orchestra or only the theme; perhaps she was hearing the
orchestration in her own mind.

She thought dimly that there had been premonitory echoes of this theme in all of Richard Halley's
work, through all the years of his long struggle, to the day, in his middle-age, when fame struck him
suddenly and knocked him out. This — she thought, listening to the symphony — had been the goal of
his struggle. She remembered half-hinted attempts in his music, phrases that promised it, broken bits
of melody that started but never quite reached it; when Richard Halley wrote this, he . . . She sat up
straight. When did Richard Halley write this?"""




In [4]:
# upload your style dataset file (need only once)
from google.colab import files
uploaded=files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving Nabokov-All.txt to Nabokov-All.txt
User uploaded file "Nabokov-All.txt" with length 9751064 bytes


In [5]:
# install dependencies (need only once)
!pip install sacremoses
!pip install gpt-2-simple
!pip install nltk

     |████████████████████████████████| 860kB 2.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=3bad420355e2c786062f5cd1017adff325c59efc5c440d821b8cb314baedae18
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
     |████████████████████████████████| 645kB 8.1MB/s 
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.6-cp36-none-any.whl size=25388 sha256=dc00ebc58df0fb72b2a68a63ac782504c92962117e8ef445e73801189ccbd32a
  Stored in directory: /root/.cache/pip/wheels/cc/e7/21/4cb10bcf085ff791a08bbd03aa3fd860f6e730f37b5dbbea28
Successfully built gpt-2-simple


In [2]:
# run at every kernel restart. System depends on tensorflow 1.14 <= and will not work for 2.0
import sys, getopt, nltk
import gc
from pathlib import Path
from nltk.tokenize import word_tokenize
from sacremoses import MosesDetokenizer
import gpt_2_simple as gpt2
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import nltk

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
# download tokenizer file (need only once)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# run at every kernel restart.
# GPT-2 released three models
# 117, 124M will train on K40 GPU, 345M needs 16Gb+ of memory (V100 instance)
model_name="117M"
#model_name="124M"
#model_name="345M"

# this is your style dataset for GPT-2 finetuning
style = "input.txt"

# content input and output
inputfile = 'content.txt'
outputfile = 'output.txt'
temperature = 1.0
minsimilarity = 0.5
minwords = 4
maxwords = 15
seedLexemes = 5
nsamples = 50

bestLexemes = []
output = ""

# get around resource exhaustion for embeddings
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

In [4]:
# run at every kernel restart.
# init tokenizer and sentence embedder
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
dt = MosesDetokenizer()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"
embed = hub.Module(module_url)
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:

# download GPT-2 model (only needed once)
gpt2.download_gpt2(model_name=model_name)




Fetching checkpoint: 1.05Mit [00:00, 302Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 103Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 336Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 199Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 530Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 164Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 177Mit/s]                                                       


In [0]:
# run at every kernel restart.
# set number of steps for finetuning or load the pretrained model

sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess)
#
# this frequently fails in collab based on resource exhaustion
# Good training results in a loss of around 0.1-0.2 (over 10K iterations)
# GPT-2 model generates coherent output on low finetuning counts with no style.

# additional traning:
# gpt2.finetune(sess, 'input.txt', model_name=model_name, restore_from = 'latest', overwrite=True, steps=1000)

# finetuning from scratch:
gpt2.finetune(sess, 'input.txt', model_name=model_name, overwrite=True, steps=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading checkpoint models/117M/model.ckpt
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:15<00:00, 15.13s/it]


dataset has 2363594 tokens
Training...
Saving checkpoint/run1/model-0
[1 | 12.89] loss=3.89 avg=3.89
[2 | 17.54] loss=4.08 avg=3.99
[3 | 22.18] loss=3.71 avg=3.90
[4 | 26.84] loss=4.00 avg=3.92
[5 | 31.46] loss=3.74 avg=3.88
[6 | 36.06] loss=3.87 avg=3.88
[7 | 40.72] loss=3.97 avg=3.89
[8 | 45.33] loss=3.93 avg=3.90
[9 | 49.93] loss=4.07 avg=3.92
[10 | 54.54] loss=3.84 avg=3.91
[11 | 59.16] loss=3.40 avg=3.86
[12 | 63.76] loss=3.83 avg=3.86
[13 | 68.40] loss=3.98 avg=3.87
[14 | 72.99] loss=3.75 avg=3.86
[15 | 77.59] loss=3.89 avg=3.86
[16 | 82.22] loss=4.26 avg=3.89
[17 | 86.82] loss=3.87 avg=3.89
[18 | 91.42] loss=3.93 avg=3.89
[19 | 96.02] loss=3.83 avg=3.89
[20 | 100.62] loss=3.71 avg=3.88
[21 | 105.23] loss=3.68 avg=3.87
[22 | 109.84] loss=4.17 avg=3.88
[23 | 114.43] loss=3.81 avg=3.88
[24 | 118.99] loss=3.92 avg=3.88
[25 | 123.57] loss=3.70 avg=3.87
[26 | 128.14] loss=3.38 avg=3.85
[27 | 132.71] loss=3.71 avg=3.85
[28 | 137.29] loss=3.74 avg=3.84
[29 | 141.91] loss=3.81 avg=3.84
[

In [0]:
# expansion formation rules: seed phrase deleted, truncation at first end 
# of sentence, the last comma, or allow to run off unclipped
# 
def cleaned(expansions, seed):

   seedbag = word_tokenize(seed)
   cutseed = dt.detokenize(seedbag[1:])
   symlen = len(cutseed)
   cleaned = []
   expansion = ""
   for exp in expansions:
      pos = exp.find(cutseed)
      if pos > 0:
         expansion = exp[pos+symlen:].strip()
      else:
         expansion = exp
      pos = expansion.find(".")
      if pos > 0:
         expansion = expansion[:pos]+"."
      else:
         pos = expansion[::-1].find(",")
         if pos > 0:
            expansion = expansion[:-pos]
      cleaned.append(expansion)
      
   return cleaned

In [0]:
def generateLexeme(sent):

      if sent == "":
         return

      words = word_tokenize(sent)

      lexeme = ""
      count = 0

      if len(words) <= maxwords:
         processLexeme(sent)

      else:
         for word in words:
            count += 1
            if (word in [";", ","] and count > minwords) or count > maxwords:
               if (len(words)-count) >  minwords:
                  processLexeme(dt.detokenize(words[:count]))
                  sentence = dt.detokenize(words[count:])
                  generateLexeme(sentence)
               else:
                  processLexeme(dt.detokenize(words))

               break

In [0]:
def processLexeme(sent):

   # always start from the first phrase
   if not bestLexemes:
      bestLexemes.append(sent)
      print("Leading output with seed phrase: {}".format(sent))
      return
    
   seed = dt.detokenize(bestLexemes[-seedLexemes:])
   print("seed: \"{}\"".format(seed))
   print("processing lexeme: \"{}\"".format(sent))
   
   expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)
   #expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=".", return_as_list=True, include_prefix=False)
   #expansions += gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=",", return_as_list=True, include_prefix=False)
   expansions = cleaned(expansions, seed)
   print("  expansion set: {}".format(expansions))
  
   messages = [sent]+expansions
   bestLexeme = sent
  
   gc.collect()

   with tf.Session(config=config) as session:
      session.run(tf.global_variables_initializer())
      session.run(tf.tables_initializer())
      message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

      corr = np.inner(message_embeddings_, message_embeddings_)
      embeddings = corr[0,1:]
      #print("embeddings: {}".format(embeddings))
      
      bestIndex = np.argmax(embeddings)
      
      if embeddings[bestIndex] >  minsimilarity:
        
         bestLexeme = expansions[bestIndex]
          
         # join expansion with future lexeme smoothly
        
         originalEnding = sent[-1]
         end = originalEnding if originalEnding in [".",",", ";"] else ""
        
         thisEnding = bestLexeme[-1]
         
         # replace punctiation at end of lexeme if original punctuation existed
         # if new sentence not ending yet, ignore the original markers
         if thisEnding in [".", ",", ";"]:
            bestLexeme = bestLexeme[:-1]+end
        
   print("  expanding with: \"{}\" bestscore: {} bestphrase: {}".format(bestLexeme, embeddings[bestIndex], expansions[bestIndex]))
   bestLexemes.append(bestLexeme)
    
   output = dt.detokenize(bestLexemes)
   Path(outputfile).write_text(output)

In [0]:
# playground cell 1: test expansion quality of the model

seed = "I have to say that we suck. Moreover, this opportunity leads us to nowhere."

expansions = gpt2.generate(sess, nsamples=10, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)

# get around a bug 
print(expansions)
expansions = cleaned(expansions,seed)
print(expansions)

In [0]:
# playground cell 2: test lexeme processor

# first lexeme always identity transform
# generateLexeme(" there never was a better place than")

bestLexemes = []
generateLexeme(" there never was a better place than")
print(bestLexemes)

In [0]:
# launch RPS from here.
# 
bestLexemes = []

# large number of samples exhausts Collab memory limit
nsamples = 100

# 0.5 is vague, 0.7 reasonably along the source, 0.8 close and 0.9 is literal
minsimilarity = 0.68

sentences = tokenizer.tokenize(input)

for sent in sentences:
   print("Input sentence: {}".format(sent))
   generateLexeme(sent)

output = dt.detokenize(bestLexemes)
Path(outputfile).write_text(output)

print(output)

   

Input sentence: She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her.
Leading output with seed phrase: She sat at the window of the train,
seed: "She sat at the window of the train,"
processing lexeme: "her head thrown back,"
  expansion set: ['her back against the wall, her eyes fixed on the ground,', 'her black leggings shaking.', 'her back against the wall.', 'trying to look through the window.', 'in the lap of the white-trousered conductor.', 'and, with her fingers pressing to her lips, she considered the train.', 'her face clutched in her lap, her thighs, her calves, her hips.', 'and presently the train departed.', 'she was wearing a black dress and had a large, white,', 'legs spread wide, her hands in the pockets of her dressing gown.', 'looking at the black top of an old church, between the pines.', 'the train was going to Paris, and when she said she would return in a few hours,', 'her hand playing with the back of her 

In [0]:
# print the input and final text (style applied to donor):

print(input)
print("------------------------------------------------------------")
print(bestLexemes)